In [1]:
import sys; sys.path.append('../../')
from ModelPipelines.ModelAnalysis import recursive_feature_elimination, log_weights_analysis, show_hyperparams, vc_dimension_check,validation_curves,learning_curves, BiasVariance
import numpy as np
from DataPreparation.DataPreparation import read_data
from utils import load_hyperparameters, save_model, get_metrics
from sklearn import svm
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
import pickle
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
mlq.start_quest("SVMBagging", table_dest="../../", log_defs=True)
model_name = 'SVMBagging'

# <font color="aqua">SVMBagging</font> Model

### Read the data

In [2]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical',split='all')
opt_params = load_hyperparameters(model_name)

### Initiate model

In [3]:
base_model_name = 'SVM'
base_opt_params = load_hyperparameters(base_model_name)
clf = mlq.l(BaggingClassifier)(base_estimator=svm.SVC(**base_opt_params), n_estimators=20, max_samples=0.7)

### Evaluate Model Bias

In [4]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.984     1.000     0.992       190
           1      0.975     0.965     0.970       201
           2      0.990     0.985     0.988       406
           3      0.999     1.000     0.999       680

    accuracy                          0.991      1477
   macro avg      0.987     0.988     0.987      1477
weighted avg      0.991     0.991     0.991      1477



### Evaluate Model Generalization

In [5]:
kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1, loo=False)

val_wf1 = kfold.get(f'10-Repeated 10-fold')[0]

10-Repeated 10-fold
0.9837


#### Save Model

In [6]:
save_model(model_name, clf)

### Tracking

In [7]:
mlq.log_metrics(train_acc, train_wf1, val_wf1)
mlq.end_quest()
mlq.show_logs("SVMBagging", table_dest="../../", last_k=10)